## Lab 10: Births

Please complete this lab by providing answers in cells after the question. Use **Code** cells to write and run any code you need to answer the question and **Markdown** cells to write out answers in words. After you are finished with the assignment, remember to download it as an **HTML file** and submit it in **ELMS**.

This assignment is due by **11:59pm on Thursday, April 21**.

In [ ]:
import numpy as np
from datascience import *


# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

# This is for linear regression
from sklearn.linear_model import LinearRegression

### North Carolina Births Data

In this lab, we will work with a dataset of births in North Carolina. A series of variables were collected, including characteristics about the mother as well as the birthweight of the baby. We are interested in what factors are associated with birthweight. We'll first look at predicting `weight` and different models we can build to predict it, then use bootstrapping to do inference with the regression models.

Let's start by exploring the dataset a little bit to see what it looks like. We'll take a look at the relationship between the number of weeks of gestation and the birthweight of the baby.

In [ ]:
ncbirths = Table.read_table('ncbirths.csv')

In [ ]:
ncbirths.show(5)

In [ ]:
ncbirths.scatter('weeks', 'weight')
plt.title('North Carolina Births Data')

Now, let's try doing an initial regression model. We want to fit a line that models the relationship between `weeks` and `weight`. 

<font color = 'red'>**Question 1. Set up the model object as `ols` and the `predictor` and `outcome` objects to run the linear regression, using `weeks` as the predictor and `weight` as the outcome. Then, fit the model and print out the slope and intercept.**</font>

In [ ]:
ols = ...

predictor = ...
outcome = ...

...

We can plot the line on top of the scatterplot to see what it looks like. 

In [ ]:
ncbirths.scatter('weeks', 'weight')
plt.title('North Carolina Births Data')
plt.plot(predictor, ols.predict(predictor) , lw=4, color='gold')

What if we wanted to try to predict the weight using the mother's `smoker` status? Since `smoker` is a categorical variable, we'd need to think about our interpretation of the slope differently, but we can still fit a regression model using this variable. However, we need to work with the data a little bit to be able to fit a model using categorical data. Essentially, what we do is treat each of the categories as a 0/1 variable. The observation has a value of 1 if it is in that category, and 0 if it is not. 

The `sklearn` package only allows for categorical variables that have been changed into dummy variables (that is, 0/1 variables). So, we're going to need to create new variables that contain the same information, except as numbers. Luckily, True/False maps onto 1/0, so we can just do comparisons. We'll use a lot of these variables later, so let's do some cleaning now.

In [ ]:
ncbirths_dummy = ncbirths.with_columns('premature', ncbirths.column('premie') == 'premie', # True if premature, False if not
                     'female', ncbirths.column('gender') == 'female', # True if female, False if not
                     'smoker', ncbirths.column('habit') == 'smoker', # True if smoker, False if not
                     'label', ncbirths.column('lowbirthweight') == 'low') # Our outcome. True if low birthweight, False if not


ncbirths_dummy.show(5)

In [ ]:
# Drop redundant rows now
ncbirths_clean = ncbirths_dummy.drop('premie', 'gender', 'habit','lowbirthweight')
ncbirths_clean.show(5)

Now that we've cleaned up our dataset, let's try doing a linear regression with a categorical predictor.

<font color = 'red'>**Question 2. Set up the predictor and outcome variables to run the linear regression, using `smoker` as the predictor and `birthweight` as the outcome.**</font>

**Hint:** This is done the same way as the linear regression above, except using `smoker` when defining `predictor`, which happens to be a categorical variable.

In [ ]:
ols = LinearRegression()

predictor = ...
outcome = ...

ols.fit(X = predictor, y = outcome)
print(ols.coef_)
print(ols.intercept_)

### Multiple Regression

You can also add additional predictor variables to the linear regression to try to better predict the outcome. This is done by simply adding more variables the `select` statement when defining the predictor. You can think about this as just adding additional terms to the equation of the line. Before, we had one intercept and one slope. Now, we'll still only have one intercept, but we'll also add in other "slopes", or coefficients with additional variables. This way, we are using multiple variables to try to predict our outcome.

In [ ]:
multiple_ols = LinearRegression()

predictor = ncbirths_clean.select('mage', 'weeks', 'female', 'smoker').rows
outcome = ncbirths_clean.column('weight')

multiple_ols.fit(X = predictor, y = outcome)
print(multiple_ols.coef_)
print(multiple_ols.intercept_)

<font color = 'red'>**Question 2. Write out the form of the equation in the model that we just ran above.**</font>

*Your answer here*

## Inference for Regression

Now that we've constructed a model to predict a baby's birthweight, we might want to do some inference on some characteristics. Consider, for example, the coefficient for `weeks` in the model above. How certain are we that there is actually a relationship between `weeks` and `weight`? Maybe we got the coefficient that we did due to chance rather than a real association between the two variables. 

We can answer this question using bootstrapping and confidence intervals. The process is the same as before:
- Take a bootstrap sample of the original data.
- Fit a new line using the bootstrap sample.
- Repeat this process many times.
- Find the confidence interval using the bootstrap results.

Let's see what this looks like with one bootstrap sample first.

In [ ]:
bootstrap_births = ncbirths_clean.sample()

bootstrap_ols = LinearRegression()
predictor = bootstrap_births.select('mage', 'weeks', 'female', 'smoker').rows
outcome = bootstrap_births.column('weight')

bootstrap_ols.fit(X = predictor, y = outcome)
print(bootstrap_ols.coef_)
print(bootstrap_ols.intercept_)

<font color = 'red'>**Question 3. Define a function called `bootstrap_slope` that has one argument `births` representing the original dataset. This function should take a bootstrap sample of `births`, fit a linear regression model with the four predictors (`mage`, `weeks`, `female`, and `smoker`), and return an array of the coefficients for the four variables.**</font>

In [ ]:
def bootstrap_slope(births):
    ...
    
    return ...

bootstrap_slope(ncbirths_clean)

<font color = 'red'>**Question 4. Use a loop to take 500 bootstrap samples and store the coefficient for each of the predictor variables within arrays.**</font>

*Hint:* Keep in mind the order in which you selected the variables to include in the model. `bootstrap_slope` should return an array with four coefficients. How would you get the individual coefficients?

In [ ]:
mage_coefs = make_array()
weeks_coefs = make_array()
female_coefs = make_array()
smoker_coefs = make_array()

for i in np.arange(500):
    coefs = bootstrap_slope(ncbirths_clean)
    mage_coefs = ...
    weeks_coefs = ...
    female_coefs = ...
    smoker_coefs = ...


We should now have four different arrays of 500 bootstrap values, one for each of our coefficients. We want to derive confidence intervals for each of these. To do this, we can define a function that gives us a confidence interval and use it to find the confidence interval for the four arrays of bootstrap coefficients.

The `confidence_interval` function has as its inputs an array of bootstrapped coefficients as well as a confidence level. It will output the left and right ends of the confidence interval as an array. 

In [ ]:
def confidence_interval(coefficients, confidence_level):
    left = percentile((100-confidence_level)/2, coefficients)
    right = percentile(100 - (100 - confidence_level)/2, coefficients)
    return make_array(left, right)

# 95% confidence interval for mother's age coefficient
confidence_interval(mage_coefs, 95)

<font color = 'red'>**Question 5. What are the 95% confidence intervals for each of the four coefficients? What would be your conclusion based on these confidence intervals?**</font>

## Prediction and Prediction Inference

We can also build confidence intervals for the prediction that we make. The process for doing this is very similar to finding the confidence interval for a coefficient:
- Take a bootstrap sample of the original data.
- Fit a new line using the bootstrap sample and calculate the prediction using that line.
- Repeat this process many times.
- Find the confidence interval using the bootstrap results.

<font color = 'red'>**Question 6. What would be the predicted weight of a baby according to our model if the mother's age was 30, the pregnancy lasted 36 weeks, the mother was not a smoker, and the baby was female?**</font>

In [ ]:
new_table = Table().with_columns('mage', ...,
                                 'weeks', ...,
                                'smoker', ...,
                                'female', ...).rows
...

 What would be the prediction interval for the prediction you found above? Let's go through the steps of generating the bootstrap prediction interval.

<font color = 'red'>**Question 7. First, define a function called `bootstrap_prediction` that takes in the original data and new x-values that you want to make predictions for. This function should take a bootstrap sample, fit a linear regression model using the same predictors as before, and return one number that represents the predicted birth weight.**</font>

*Hint:* There hasn't been much code provided here, but look back at what we've done before and think about what you would need to do. This function should be similar to the `bootstrap_slope` function, but the output will be different.

In [ ]:
def bootstrap_prediction(births, new_x):
    ...
    
    return ...

bootstrap_prediction(ncbirths_clean, new_table)

<font color = 'red'>**Question 8. Find the 95% prediction interval. Use 500 iterations in the loop, and assign the 500 bootstrap predictions to `predictions`. Assign the left endpoint of the confidence interval to `left` and the right endpoint to `right`.**</font>

*Hint:* Again, there hasn't been much code provided here, but look back at what we've done before and think about what you would need to do. You can feel free to reuse the `confidence_interval` function to calculate the endpoints of the interval.

In [ ]:
predictions = ...

...

left = ...
right = ...

print('The prediction interval is:', left, ',', right)

We can graph the bootstrap predictions and look at the prediction interval as we've done with bootstrap values in the past.

In [ ]:
Table().with_columns('Prediction', predictions).hist()
left = percentile(2.5, predictions)
right = percentile(97.5, predictions)
plt.plot([left, right], [0, 0], color='yellow', lw=10, zorder=1)